In [34]:
import os
import time

import cv2
import matplotlib.pyplot as plt
import natsort
import numpy as np
import pandas as pd
from opto import Opto
from scipy.interpolate import interp1d
from tqdm import tqdm
from ximea import xiapi

%matplotlib inline
import seaborn as sns
from tqdm.contrib.concurrent import process_map

In [2]:
def detect_blur_fft(image, size=60, threshold=10):
    (h, w) = image.shape
    (cx, cy) = (int(w / 2.0), int(h / 2.0))
    fft = np.fft.fft2(image)
    fftShift = np.fft.fftshift(fft)

    fftShift[cy - size : cy + size, cx - size : cx + size] = 0
    fftShift = np.fft.ifftshift(fftShift)
    recon = np.fft.ifft2(fftShift)

    magnitude = 20 * np.log(np.abs(recon))
    mean = np.mean(magnitude)

    return (mean, mean <= threshold)


def process_file(args):
    file, folder = args
    current = os.path.splitext(os.path.basename(file))[0]
    image_path = os.path.join(folder, file)

    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        return (current, None)

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Failed to read image: {image_path}")
        return (current, None)

    mean, blurry = detect_blur_fft(image, size=60, threshold=10)
    return (current, mean)


def is_filename_numeric(filename):
    name, ext = os.path.splitext(filename)
    return name.isdigit()


def find_best_focus_point(folder: str):
    files = os.listdir(folder)
    files = natsort.natsorted([f for f in files if f.endswith(".png")])
    # files = [file for file in files if is_filename_numeric(file)]

    args = [(file, folder) for file in files]
    results = process_map(process_file, args, max_workers=os.cpu_count())

    files_values = []
    fft_values = []
    for file, fft_value in results:
        if fft_value is not None:
            files_values.append(file.split("_")[0])
            fft_values.append(fft_value)

    df = pd.DataFrame({"current": files_values, "fft": fft_values})

    df.to_csv(f"{folder}/fft_values.csv", index=False)

    max_fft_for_current = files_values[np.argmax(fft_values)]

    fig = plt.figure()
    sns.lineplot(df, x="current", y="fft")
    plt.title(f"{max_fft_for_current}")
    plt.savefig(f"{folder}/fft_values.png")
    plt.close(fig)

    return df

In [3]:
def open_controller(dev="/dev/ttyACM0"):
    o = Opto(dev)
    o.connect()
    return o


def open_camera():
    cam = xiapi.Camera()
    cam.open_device()
    cam.set_exposure(1500)
    return cam


def find_optimal_focus_for_liquid_lens(
    position: float, current: int, move_range: int = 10, repeats: int = 10
):
    o = open_controller()
    cam = open_camera()

    img = xiapi.Image()

    cam.start_acquisition()

    current_array = np.arange(current - move_range, current + move_range, 1)

    # create calibration folder if it doesn't exist
    try:
        save_path = f"/home/buchsbaum/calibration/{position:.1f}/"
        os.makedirs(save_path)
    except FileExistsError:
        pass

    for _, curr in enumerate(tqdm(current_array, desc="Capturing images")):
        o.current(curr)
        time.sleep(0.5)

        for j in range(repeats):
            cam.get_image(img)
            data = img.get_image_data_numpy()

            # save grayscale image to disk
            cv2.imwrite(f"{save_path}/{curr}_{j}.png", data)

    cam.stop_acquisition()
    cam.close_device()
    o.close(soft_close=True)
    return save_path

In [30]:
position = 35
mid_current = 250
move_range = 10

In [31]:
save_path = find_optimal_focus_for_liquid_lens(position, mid_current, move_range)

xiAPI: ---- xiOpenDevice API:V4.29.04.00 started ----
xiAPI: EAL_IF_xiFAPI_Top::InitializeDevice sn:BLMID2407000 name:CB160MG-LX-X8G3
xiAPI: Bandwidth measurement - disabling frame buffer before measurement
xiAPI: Bandwidth measurement - frame buffer disabled OK
xiAPI: XiApiToGentlParamModel Auto bandwidth measurement finished (7035MBps). Safe limit set to: 5628MBps
xiAPI: SAL_Common_SetAcquisitionFrameRate framerate is limited in freerun by bandwidth to 349.5FPS
xiAPI: FGTL_SetParam_to_CAL error from CAL: -10009, addr:x201380
xiAPI: ---- Device opened. Model:CB160MG-LX-X8G3 SN:BLMID2407000 FwF1:22.33 API:V4.29.04.00 ----
xiAPI: xiFAPI_Device::AllocateBuffers Allocating buffers. Count:33 OneBuff:15764 KiB All:512 MiB Frm:x1080001
xiAPI: SAL_Common_SetAcquisitionFrameRate framerate is limited in freerun by bandwidth to 349.5FPS
Capturing images: 100%|██████████| 20/20 [00:48<00:00,  2.42s/it]
xiAPI: xiCloseDevice


In [32]:
focus_df = find_best_focus_point(f"/home/buchsbaum/calibration/{position:.1f}/")

100%|██████████| 200/200 [00:40<00:00,  4.92it/s]


In [45]:
X = np.array([0.0, 7.5, 15.0, 20, 29.0, 35.0])
y = np.array([97, 121, 144, 162, 208, 252])


current_interpolation_function = interp1d(X, y, kind="linear", fill_value="extrapolate")